In [24]:
import requests
from bs4 import BeautifulSoup as bs
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import string
import re
import pandas as pd 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [25]:
def parse_pages_cotopaxi(pages):
    names = []
    stars = []
    short_reviews = []
    full_reviews = []
    collapsed_reviews = []
    
    for page in pages:
        divs = page.find_all("div", class_="yotpo-review")
        divs += page.find_all("div", class_="yotpo-review yotpo-regular-box yotpo-regular-box-filters-padding")
        
        for div in divs:
            names.append(div.find("span", attrs={"class":"y-label yotpo-user-name yotpo-font-bold pull-left"}))
            stars.append(len(div.find_all("span", attrs={"class":"yotpo-icon yotpo-icon-star rating-star pull-left"})))
            short_reviews.append(div.find("div", attrs={"class":"content-title yotpo-font-bold"}))
            full_reviews.append(div.find("div", attrs={"class":"content-review"}))
            collapsed_reviews.append(div.find("p", attrs={"class":"rest-content-collapsed"}))
    return (names, stars, short_reviews, full_reviews, collapsed_reviews)

def parse_pages_patagonia(pages):
    names = []
    stars = []
    short_reviews = []
    full_reviews = []
    collapsed_reviews = []
    
    for page in pages:
        divs = page.find_all("div", class_="yotpo-review")
        divs += page.find_all("div", class_="yotpo-review yotpo-regular-box yotpo-regular-box-filters-padding")
        
        for div in divs:
            names.append(div.find("span", attrs={"class":"y-label yotpo-user-name yotpo-font-bold pull-left"}))
            stars.append(len(div.find_all("span", attrs={"class":"yotpo-icon yotpo-icon-star rating-star pull-left"})))
            short_reviews.append(div.find("div", attrs={"class":"content-title yotpo-font-bold"}))
            full_reviews.append(div.find("div", attrs={"class":"content-review"}))
            collapsed_reviews.append(div.find("p", attrs={"class":"rest-content-collapsed"}))
    return (names, stars, short_reviews, full_reviews, collapsed_reviews)

def parse_pages_loveyourmelon(pages):
    names = []
    stars = []
    short_reviews = []
    full_reviews = []
    collapsed_reviews = []
    
    for page in pages:
        divs = page.find_all("div", class_="yotpo-review yotpo-regular-box")
        divs += page.find_all("div", class_="yotpo-review yotpo-regular-box yotpo-regular-box-filters-padding")
        
        for div in divs:
            names.append(div.find("label", attrs={"class":"y-label yotpo-user-name yotpo-font-bold pull-left"}))
            stars.append(len(div.find_all("span", attrs={"class":"yotpo-icon yotpo-icon-star rating-star pull-left"})))
            short_reviews.append(div.find("div", attrs={"class":"content-title yotpo-font-bold"}))
            full_reviews.append(div.find("div", attrs={"class":"content-review"}))
            collapsed_reviews.append(div.find("p", attrs={"class":"rest-content-collapsed"}))
    return (names, stars, short_reviews, full_reviews, collapsed_reviews)

def parse_pages_wandererbracelets(pages):
    names = []
    stars = []
    short_reviews = []
    full_reviews = []
    collapsed_reviews = []
    
    for page in pages:
        divs = page.find_all("div", class_="okeReviews-reviews-review")
        
        for div in divs:
            names.append(div.find("span", attrs={"class":"okeReviews-review-reviewer-profile-name"}))
            stars.append(div.find_all("span", attrs={"class":"okeReviews-starRating-indicator-layer okeReviews-starRating-indicator-layer--foreground"}))
            stars[-1] = int(re.sub(r'[^0-9.]+', r'', str(stars[-1]))) // 20
            short_reviews.append(div.find("h2", attrs={"class":"okeReviews-review-main-heading"}))
            full_reviews.append(div.find("div", attrs={"class":"js-okeReviews-reviewContent okeReviews-review-main-content-body"}))
            collapsed_reviews.append(div.find("p", attrs={"class":"rest-content-collapsed"}))
    return (names, stars, short_reviews, full_reviews, collapsed_reviews)

# PARAMETERS

In [ ]:
# Начальная страница комментов (название сайта используется для выбора парсера)
start_url = 'https://loveyourmelon.com/collections/scarves/products/smoke-speckled-infinity-scarf-black-leather-patch'
# на сколько страниц нужно перелестнуть комментарии
number_pages = int(684 / 5)
# какой селектор отвечает за переход на следующую страницу комментариев
arrow_selector = '.yotpo-icon-right-arrow'
close_selector = '.privy-dismiss-content'
#arrow_selector = '.okeReviews-reviews-showMore'
#close_selector = '.design-layer-editable'

In [43]:
chrome_options = Options()
chrome_options.add_argument("--headless") # Opens the browser up in background

driver = Chrome("chromedriver.exe")

pages = []
with Chrome(options=chrome_options) as browser:
    driver.get(start_url)
    request = driver.execute_script('return document.body.innerHTML')
    pages.append(bs(''.join(request), "html.parser"))
    
    time.sleep(2)    
    
    for i in range(number_pages):
        try:
            driver.find_element_by_css_selector(close_selector).click()
        except:
            pass
        driver.find_element_by_css_selector(arrow_selector).click()
        time.sleep(2)
        request = driver.execute_script('return document.body.innerHTML')
        pages.append(bs(''.join(request), "html.parser"))

Exception ignored in: 

WebDriverException: Message: chrome not reachable
  (Session info: chrome=79.0.3945.130)


In [ ]:
len(pages)

In [26]:
names = []
short_reviews = []
full_reviews = []
collapsed_reviews = []
stars = []

if "www.cotopaxi.com" in start_url:
    print('there')
    (names, stars, short_reviews,
    full_reviews, collapsed_reviews) = parse_pages_cotopaxi(pages)

if "www.patagonia.com" in start_url:
    print('there')
    (names, stars, short_reviews,
    full_reviews, collapsed_reviews) = parse_pages_patagonia(pages)
    
if "loveyourmelon.com" in start_url:
    print('there')
    (names, stars, short_reviews,
    full_reviews, collapsed_reviews) = parse_pages_loveyourmelon(pages)
    
if "wandererbracelets.com" in start_url:
    print('there')
    (names, stars, short_reviews,
    full_reviews, collapsed_reviews) = parse_pages_wandererbracelets(pages)

there


In [27]:
names[0]

<span aria-hidden="true" class="okeReviews-review-reviewer-profile-name">Robert J P.</span>

In [29]:
stars[0]

5

In [30]:
i = 0

while (i < len(names)):
    if names[i].text == '':
        del names[i]
        del stars[i]
        del short_reviews[i]
        del full_reviews[i]
        del collapsed_reviews[i]
        
    else:
        i += 1

# FOR CHECK. Names the number of names must match the number of reviews.

In [31]:
len(names)

330

In [32]:
print(names[0].text)
print(stars[0])
print(short_reviews[0].text)
print(full_reviews[0].text)
if not collapsed_reviews[0] is None:
    print(collapsed_reviews[0].text)

Robert J P.
5

Bracelet


Well packaged, ready to gift.



In [33]:
for i in range(len(names)):
    names[i] = names[i].text
    short_reviews[i] = short_reviews[i].text
    full_reviews[i] = full_reviews[i].text
    if collapsed_reviews[i] is None:
        collapsed_reviews[i] = ''
    else:
        collapsed_reviews[i] = collapsed_reviews[i].text

# Create dataframe

In [34]:
reviews = pd.DataFrame(zip(names, stars, short_reviews, full_reviews, collapsed_reviews), 
             columns = ['Name', 'Star', 'Short review', 'Full review', 'Collapsed_reviews'])

In [35]:
count = 0

for i in range(len(names)):
    for j in range(i + 1, len(names)):
        if full_reviews[i] == full_reviews[j] and names[i] == names[j] and short_reviews[i] == short_reviews[j]:
            count += 1
print(len(names) - count)

-770


In [36]:
reviews

,Name,Star,Short review,Full review,Collapsed_reviews
0,Robert J P.,5,\nBracelet\n,"\nWell packaged, ready to gift.\n",
1,Karen,5,\nLovely gift\n,\nMy boyfriend was so happy about this gift\n,
2,Lizzie B.,5,\nAmazing gift\n,\nLove this gift! Love that it supports artisa...,
3,Mark A. C.,5,\nWow!\n,\nThe coordinates I used were for the home pla...,
4,Erica A.,5,\nOrdering process was seamless!\n,\nLove the product!\n,
...,...,...,...,...,...
325,Jennifer B.,5,\nCustom coordinates bracelet\n,\nI absolutely love my bracelet!!\n,
326,Federica,5,\nPerfect!\n,\nI bought it for my 1st anniversary for the l...,
327,Malinda S.,5,\nAmazing\n,\nThe bracelet turned out incredible. My husba...,
328,Shawn C.,5,\nBracelets\n,\nGot these made up for my girlfriend and I an...,


# Save dataframe

In [37]:
reviews.to_csv("BLACK SPECKLED BEANIE.csv")